> For more information how to run the Jupyter notebooks and our overall philosophy regarding interactive docs, read more here. (TODO)

# `ipfs-log`

In [1]:
const Log = require("../src/log") // const Log = require("ipfs-log") elsewhere

## Intro

The `Log` provided by the `ipfs-log` package is an implementation of a Conflict-Free Replicated Data Type (CRDT) that utilizes the Interplanetary File System (IPFS) as a storage backend. The functionality in this package forms the backbone of [`orbit-db`](https://github.com/orbitdb/orbit-db).

## Table of Contents

1. [Top](#)
  1. [Intro](#Intro)
  2. [Nomenclature and Concepts](#Nomenclature-and-Concepts)
2. [Prerequisites](#Prerequisites)
  1. [IPFS Node](#IPFS-Node)
  2. [Access Controller](#Access-Controller)
  3. [Identity](#Identity)
3. [Usage](#Usage)
  1. [Creating Logs](#Creating-Logs)
  3. [Manipulating Logs](#Manipulating-Logs)
  4. [Joining Logs](#Joining-Logs)
4. [API Documentation](#API-Documentation)
  1. [Log](#Log)

## Nomenclature and Concepts

### Conflict-Free Replicated Data Type (CRDT)

A **Conflict-Free Replicated Data Type** is a type of log that solves the problem of locally storing, and ultimately merging, distrubuted data sets to other distributed data sets<sup>1</sup>. It allows users to perform operations on *local* databases with the intent of merging or joining those data with the data stored on the devices of other peers in the network.

### Lamport Clock

To achieve successful merging - merging that is properly *associative* - entries are timestamped with something called a Lamport Clock<sup>2</sup>. The timestamp of each entry is a pair of values: a _logical clock_ counter of the entry (as opposed to _wall clock_), and an identifier of the user or device that generated the entry.

In the case of `ipfs-log`, the identifier is the public key of the IPFS node where the entries are initially generated.

```javascript
// Lamport Clock Object
{
  id: '042750228c5d81653e5142e6a56d55...e5216b9a6612dbfc56e906bdbf34ea373c92b30d7',
  time: 0
}
```

### Heads and Tails

Heads and Tails are important concepts in terms of CRDTs, and many people require a bit of explanation before fully understanding the concept and its implications.

### Heads

The **head** of a log is an entry that is not referenced by any other entry. Practically speaking, these are the latest entries being appended to a log or logs.

This is best understood by example, observing how the heads change over time. In the following examples, circles are entries, green circles are heads, and arrows denote the pointers contained in the entry, to the previous record.

Let's start with the simplest example - a single user writing entries to a single log.

![Single Node CRDT over time, Simplest Example](./single-node-log-over-time.png)

However, we are not in the business of single-device / single-user logs, so let's imagine the following scenario in an attempt to find the least-complex, but still _complete_ example of how the heads would work over time.

First, in plain words and some pseudocode:

1. User 1 starts a Log (`log1 = new Log`)
2. User 2 starts a Log (`log2 = new Log`)
3. User 1 adds two entries to the log (`log1.append`)
4. User 2 merges that log with their own (`log2.join(log1)`)
5. User 1 adds two more entries (`log1.append`)
6. User 2 adds an entry (`log2.append`)
7. User 2 merges the log again (`log2.join`)
8. User 2 adds one more entry (`log2.append`)

Now in a diagram:

![Multiple Node CRDT over time, Least-Complex Example](./multiple-nodes-log-over-time.png)

We can then see how it's possible that a CRDT may have more than one head entry (maybe hundreds) and how those entries change over time with multiple users.

### Tails

A CRDT of any size can be stored in IPFS. However, When performing computations on the data, it needs to be loaded into an "input array" (i.e. subset of the log) that exists in a finite memory space. The **tails** of such a log point to entries that are not in the input array.

For our example, let's imagine a log with hundreds of millions of entries. You don't have access to a supercomputing center so tt's not feasible to load the log into memory. Thus, we use a partial traversal of the log, the **tails** of which contain the pointers to the next records to be traversed, if we so choose.

This concept is visualized below, with the dim entries signifying non-traversed, and the orange entries signifying tails.

![Tails Example](./tails-example.png)

### G-Set

`ipfs-log` specifically uses a G-Set CRDT, which in practice means append-only with no deletetion.

```javascript
class GSet {
  constuctor (values) {}
  append (value) {}
  merge (set) {}
  get (value) {}
  has (value) {}
  get values () {}
  get length () {}
}
```

#### References

1. https://citemaster.net/get/10b50274-7bc5-11e5-8aa1-00163e009cc7/p558-lamport.pdf
2. https://hal.inria.fr/inria-00555588

## Prerequisites

For a minimum viable `ipfs-log`, you need three things: a running IPFS node, an access controller, and an identity.

### IPFS Node

For our examples, we'll switch between a node.js [`js-ipfs`](https://github.com/ipfs/js-ipfs) instance, and a go node using [`js-ipfs-api`](https://github.com/ipfs/js-ipfs-api).

In [ ]:
const IPFS = require("ipfs")
const IpfsApi = require("ipfs-api")

In [ ]:
// Default config
var ipfs = new IPFS()
var ipfsapi = new IpfsApi("localhost", 5001)

^ Always a good sign.

### Access Controller

TODO: Details

In [ ]:
const AccessController = require('./src/default-access-controller')

In [ ]:
var testACL = new AccessController()

### Identity

TODO: Details


In [ ]:
const IdentityProvider = require('orbit-db-identity-provider')
const Keystore = require('orbit-db-keystore')

In [ ]:
const keystore = Keystore.create("./test/fixtures/keys")
const identitySignerFn = async (id, data) => {
    const key = await keystore.getKey(id)
    return keystore.sign(key, data)
}

let testIdentityA, testIdentityB;
(async () => { 
    testIdentityA = await IdentityProvider.createIdentity(keystore, 'userA', identitySignerFn)
    testIdentityB = await IdentityProvider.createIdentity(keystore, 'userB', identitySignerFn)
    testIdentityC = await IdentityProvider.createIdentity(keystore, 'userC', identitySignerFn)
})()

## Usage

With the following, we can create a minimum viable log:
* a working IPFS node
* access controller
* an identity

The full signature is 

```javascript
new Log(ipfs, access, identity, [logId], [entries], [heads], [clock])
```

We'll more into the optional params now. For now, let's create a couple "minimum viable" logs.

### Creating Logs

In [ ]:
var log = new Log(ipfs, testACL, testIdentityA)   
var log2 = new Log(ipfsapi, testACL, testIdentityB)

If you don't supply a `logId`, the current javascript timestamp will be used.

In [ ]:
log.id

If no log `entries` are specified, the log's `length` will be 0.

In [ ]:
log2.length

`ipfs-log` uses Lamport Clocks, which are a type of vector clock. By tracking both the ID and timestamp of each entry, logs can be merged with other logs and still produce a unique, sorted, set of entries to be processed. This data type is compatible with any "pure" function. 

In [ ]:
log.clock

In [ ]:
log2.clock

### Manipulating Logs

Now that we have our logs, let's create some entries!

TODO: Entry Signature

In [ ]:
const Entry = require("./src/entry")
const Clock = require('./src/lamport-clock')

### Adding entries at log creation


By utilizng the first of our optional params, we can create a log with entries right from the start. Observe how the length and clock values change. Please note that with this method you have to _manually_ set the clock values for each entry.

In [ ]:
let log3;

(async() => {
    var entry1 = await Entry.create(ipfs, testIdentityC, 'C', 'entry1', [], new Clock('C', 0))
    var entry2 = await Entry.create(ipfs, testIdentityC, 'C', 'entry2', [], new Clock('C', 1))
    log3 = new Log(ipfs, testACL, testIdentityC, null, [entry1, entry2])
})()

In [ ]:
log3.length

In [ ]:
log3.clock

### Adding entries after log creation

Note that the clock in the entries will be ignored and are therefor not necessary for this method.

In [ ]:
(async () => {
    var entry3 = await Entry.create(ipfs, testIdentityC, 'C', 'entry3', [])
    var entry4 = await Entry.create(ipfs, testIdentityC, 'C', 'entry4', [])
    var entry5 = await Entry.create(ipfs, testIdentityC, 'C', 'entry5', [])
    log3.append(entry3)
    log3.append(entry4)
    log3.append(entry5)
})()

In [ ]:
log3.length

In [ ]:
log3.clock

### Joining Logs

TODO: Show the magic